# Exemplo de extração de dados de páginas web

# https://www.vivareal.com.br/venda/?pagina=5

Este notebook extrai os dados de todos os [imóveis a venda no site VivaReal](https://www.vivareal.com.br/venda/) e escreve os dados extraidos num banco de dados (SQLIte).

![cartao-vivareal.png](cartao-vivareal.png)

Na última verificação, o site tinha 277 páginas de imóveis, 36 imóveis por página. Se deixá-lo rodar integralmente, teremos no final quase 10000 imóveis.

Notebook escrito por Avi Alkalay <<DataScientist@digitalhouse.com>>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3

In [2]:
# 2 opções de URL
urlBrasil = "https://www.vivareal.com.br/venda/?pagina={npagina}"
urlPinheiros = "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-oeste/pinheiros/apartamento_residencial/?pagina={npagina}"

# Vamos trabalhar com esta:
url=urlBrasil

# Para enganar o site, permutaremos entre 2 opções de assinaturas de browser.
# Peguei de https://developers.whatismybrowser.com/
userAgents=[
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/74.0.3729.157 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15"
]

dbfile = 'VivaReal.db'
tabela = 'venda'

## Análise inicial de cada elemento da página

Este trecho serve para testarmos a extração de cada elemento desejado, seja nome do imóvel, preço etc. Quando compreendido, movemos o trecho de código finalizado para o bloco mais adiante no notebook, em que o scrapping é feito em lote.

### Passo 1: Traz o HTML do site

Use o `requests` para fazer uma operação HTTP GET e obter o HTML da página.

In [3]:
url

'https://www.vivareal.com.br/venda/?pagina={npagina}'

In [4]:
url.format(npagina=1)

'https://www.vivareal.com.br/venda/?pagina=1'

In [5]:
doc = requests.get(url.format(npagina=1), headers={"User-agent": userAgents[1]})

Vamos verificar o que o webserver retornou olhando só os 700 primeiros bytes (variavel `olhadinha`).

In [6]:
olhadinha=700
print(str(doc.content[:olhadinha]) + '…')

b' <!DOCTYPE html> <html lang="pt-BR" prefix="og: https://ogp.me/ns#"> <head> <meta name="msvalidate.01" content="5DB2670D7BB1D1D3E36C46F7C3D59380"> <meta name="omniverify" content="omni2c019e0"> <meta property="og:url" content="https://www.vivareal.com.br/venda/"> <meta property="og:type" content=""> <meta property="og:image" content=""> <meta property="og:image:type" content="image/jpeg"> <meta property="og:image:secure_url" content="">  <meta property="og:title" content="Comprar Im\xc3\xb3veis - Oferta de im\xc3\xb3veis \xc3\xa0 venda - Viva Real"> <meta property="og:description" content="Mais de 5.384.453 im\xc3\xb3veis para comprar no Brasil? No Viva Real voc\xc3\xaa acha a maior oferta de im\xc3\xb3veis \xc3\xa0 venda nas melhor'…


In [7]:
doc.status_code

200

In [8]:
img=requests.get('https://resizedimgs.vivareal.com/crop/360x240/vr.images.sp/4a00f89baa208f998c73023332121ef1.jpg', headers={"User-agent": userAgents[1]})

In [9]:
img.status_code

200

In [10]:
print(str(img.content[:olhadinha]) + '…')

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xdb\x00\x84\x00\n\x07\x07\x08\x07\x06\n\x08\x08\x08\x0b\n\n\x0b\x0e\x18\x10\x0e\r\r\x0e\x1d\x15\x16\x11\x18#\x1f%$"\x1f"!&+7/&)4)!"0A149;>>>%.DIC<H7=>;\x01\n\x0b\x0b\x0e\r\x0e\x1c\x10\x10\x1c;("(;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\xff\xc2\x00\x11\x08\x00\xf0\x01h\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x001\x00\x00\x02\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x00\x03\x04\x05\x06\x07\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\xff\xda\x00\x0c\x03\x01\x00\x02\x10\x03\x10\x00\x00\x00\xee\xe3\xe9\xf3{c\xd3\xed\xcb\xae^\xf4\x93\x96\xa4\x90\x92BI\x0f3\xc4\xf4<N\xb9\xe7\xd5\xa2\x88\xa6\xbbk\xcd\xad]3Q]EWYT0\x140\x80\x08\x0006%\x80\xae\xda\x8ez\xb0\x95\x03\x08Eu\x84\x0c\x04\x86\x1fN\xe3\xf78^\xbe~\xab^=1\xe8d\x9c\xb5$\x84\x92\x12Hq8~\x83\x87\xd7\x1c\xca5g\x96\x8a\xae\xaf6\xa4\xb13Q]EV\x12\xa8a\n\x18\n\x18\x000\x86uz\x14i\xcabW\\\xd5V\x02\x0

### Passo 2: Analiza o texto HTML

A biblioteca `BeautifulSoup` tem a capacidade de analizar o HTML entregue pelo site (que não é nada mais que texto corrido, como vimos), e convertê-lo em DOM (document object model). O DOM é o mesmo documento HTML só que todas as tags e sua hierarquia foram identificadas ao ponto de podermos fazer buscas por tags e attributos de seus tags.

In [11]:
analizador = BeautifulSoup(doc.content, 'html.parser')

A variável `analizador` agora contém o DOM do texto retornado pelo web server. Então agora usamos ela para buscar tags específicas.

Usamos o _inspector_ do browser para entender a estrutura do documento que o HTMLeiro da VivaReal concebeu. Descobrimos que os imóveis estão publicados assim:

```html
<div id="12345">
    <div class="js-card-selector">
        ...mais tags sobre o imóvel...
    </div>
</div>

<div id="54321">
    <div class="js-card-selector">
        ...mais tags sobre o imóvel...
    </div>
</div>
```

Cada imóvel está contido dentro de seu respectivo `<div id="....">`, mas não temos nada específico e genérico nesta tag para selecioná-la. Já a tag imediatamente interior, `<div class="js-card-selector">`, é idêntica para cada imóvel e contém a classe `js-card-selector`, o que a torna uma ótima candidata para ser selecionada.

Então abaixo criaremos uma lista chamada `imoveis` que contém todos os trechos DOM que respeitam o nosso filtro: `<div>`s com classe `js-card-selector`.

In [12]:
analizador

 <!DOCTYPE html>
 <html lang="pt-BR" prefix="og: https://ogp.me/ns#"> <head> <meta content="5DB2670D7BB1D1D3E36C46F7C3D59380" name="msvalidate.01"/> <meta content="omni2c019e0" name="omniverify"/> <meta content="https://www.vivareal.com.br/venda/" property="og:url"/> <meta content="" property="og:type"/> <meta content="" property="og:image"/> <meta content="image/jpeg" property="og:image:type"/> <meta content="" property="og:image:secure_url"/> <meta content="Comprar Imóveis - Oferta de imóveis à venda - Viva Real" property="og:title"/> <meta content="Mais de 5.384.453 imóveis para comprar no Brasil? No Viva Real você acha a maior oferta de imóveis à venda nas melhores imobiliárias do Brasil." property="og:description"/> <meta content="Mais de 5.384.453 imóveis para comprar no Brasil? No Viva Real você acha a maior oferta de imóveis à venda nas melhores imobiliárias do Brasil." name="description"/> <title> Comprar Imóveis - Oferta de imóveis à venda - Viva Real </title> <link href="htt

In [13]:
imoveis = analizador.find_all('div', class_="js-card-selector")

Vamos dar uma olhadinha no que tem dentro de cada ítem do array:

In [14]:
len(imoveis)

36

In [15]:
for i in imoveis[:5]:
    print(str(i).replace("\n\n\n","").replace("   ","")[:200] + "…\n")

<div class="js-card-selector"> <article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <div class="p…

<div class="js-card-selector"> <article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <div class="p…

<div class="js-card-selector"> <article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <div class="p…

<div class="js-card-selector"> <article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <div class="p…

<div class="js-card-selector"> <article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <

In [16]:
len(imoveis)

36

### Passo 3: Monto receitas para extrair cada dado que desejo

Ótimo. Agora olhando a estrutura de tags e atributos, uso seletores para isolar exatamente o dado que desejo. Após isolá-lo, ainda forço ele a passar por filtros para eliminar inutilidades como espaços (uso `strip()`), prefixos como "R$ " (uso `replace("R$ ","")`) e ajusto o ponto decimal de números (uso `replace(".","")`) para em seguida convertê-los de texto para números (uso `int()`).

É muito importante para mim também obter o ID do imóvel. Como esta informação está num atrbuto do `<div>` pai, uso o seletor `parent`.

In [50]:
# Título do anúncio
print(imoveis[0].find("span",class_="property-card__title").contents[0].strip())

Apartamento com 2 Quartos à Venda, 86m²


In [44]:
# Endereço
print(imoveis[0].find("span",class_="property-card__address").contents[0].strip())

Rua Demóstenes, 606 - Campo Belo, São Paulo - SP


In [45]:
# Preço limpo e transformado em inteiro
int(imoveis[0].find("div",class_="property-card__price").find("p").contents[0].strip().replace("R$ ","").replace(".",""))

1200000

In [20]:
# Telefone
try:
    print(imoveis[0].find("a",class_="property-card__contact--phone").get('href').replace("tel:","+55"))
except:
    pass

In [21]:
# Texto descritivo
try:
    print(imoveis[0].find('ul',class_='property-card__details').text.strip())
except:
    pass

201   m²     3   Quartos     4   Banheiros     2   Vagas


In [22]:
print(int(imoveis[0].find('ul',class_='property-card__details').find('span',class_='property-card__detail-area').text.strip()))

201


In [46]:
# imoveis[0].find('div',class_='property-card__description').contents[0].strip()

# ID do imovel
print(imoveis[0].parent.get('id'))

2537788782


## Extração em massa

Depois da análise acima, podemos criar nosso loop que consome todo o site. Página por página, imóvel por imóvel, dado por dado.

In [24]:
# são 277 páginas de imóveis a venda no total
# paginas=277

# mas a títlo de exemplo, usaremos somente 20 páginas:
paginas=20

In [25]:
for x in range(1,paginas+1):
    print(x)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [26]:
url

'https://www.vivareal.com.br/venda/?pagina={npagina}'

In [51]:
# em rows guardo uma lista temporária de dicts de dados capturados, 1 dict por imóvel
imóveis=[]

for pagina in range(1,paginas+1):
    print("Estou aqui: " + url.format(npagina=pagina))
    
    # pega a página do site pela internet
    doc = requests.get(url.format(npagina=pagina), headers={"User-agent": userAgents[1]})
    
    # analiza o HTML
    analizador = BeautifulSoup(doc.content, 'html.parser')
    
    # extrai somente a lista de imóveis (em HTML) usando o seletor descoberto no código da página
    imoveis = analizador.find_all('div', class_="js-card-selector")
    
    for unidade in imoveis:
        uni={}
        # extrai dado por dado segundo seus seletores...
        
        # O ID:
        uni['id']    = unidade.parent.get('id')
        
        # O título/nome:
        uni['nome']  = unidade.find("span",class_="property-card__title").contents[0].strip()
        
        # O endereço:
        uni['ende']  = unidade.find("span",class_="property-card__address").contents[0].strip()
        
        # A metragem
        uni['metragem']=int(unidade.find('ul',class_='property-card__details').find('span',class_='property-card__detail-area').text.strip())
        
        # O preço tem uma pegadinha
        try:
            # Na maioria dos anuncios o preço está na posição 0.
            # Mas em alguns, o preço aparece como "a partir de" e causa um erro aqui.
            uni['preco'] = int(unidade.find("div",class_="property-card__price").find("p").contents[0].strip().replace("R$ ","").replace(".",""))
        except ValueError:
            # Então o meu tratamento de erro para preços "a partir de" é tentar capturá-lo na posição 2:
            try:
                uni['preco'] = int(unidade.find("div",class_="property-card__price").contents[2].strip().replace("R$ ","").replace(".",""))
            except IndexError:
                # Mas há um outro tipo de problema, onde o preço aparece "sob consulta", e causa um "IndexError". Trato assim:
                uni['preco'] = -1

        # O telefone capturo manipulando a URL e dando uma internacionalizada básica com "+55":
        try:
            uni['tel']   = unidade.find("a",class_="property-card__contact--phone").get('href').replace("tel:","+55")
        except:
            pass
        
        # A descrição:
        try:
            uni['desc']  = unidade.find('ul',class_='property-card__details').text.strip()
        except:
            pass

        # No final deste loop, o dict uni contém os dados de 1 imóvel, aí adiciono-o a uma lista de imóveis
        imóveis.append(uni)

Estou aqui: https://www.vivareal.com.br/venda/?pagina=1
Estou aqui: https://www.vivareal.com.br/venda/?pagina=2
Estou aqui: https://www.vivareal.com.br/venda/?pagina=3
Estou aqui: https://www.vivareal.com.br/venda/?pagina=4
Estou aqui: https://www.vivareal.com.br/venda/?pagina=5
Estou aqui: https://www.vivareal.com.br/venda/?pagina=6
Estou aqui: https://www.vivareal.com.br/venda/?pagina=7
Estou aqui: https://www.vivareal.com.br/venda/?pagina=8
Estou aqui: https://www.vivareal.com.br/venda/?pagina=9
Estou aqui: https://www.vivareal.com.br/venda/?pagina=10
Estou aqui: https://www.vivareal.com.br/venda/?pagina=11
Estou aqui: https://www.vivareal.com.br/venda/?pagina=12
Estou aqui: https://www.vivareal.com.br/venda/?pagina=13
Estou aqui: https://www.vivareal.com.br/venda/?pagina=14
Estou aqui: https://www.vivareal.com.br/venda/?pagina=15
Estou aqui: https://www.vivareal.com.br/venda/?pagina=16
Estou aqui: https://www.vivareal.com.br/venda/?pagina=17
Estou aqui: https://www.vivareal.com.br/

Processei todos os imóveis de todas as páginas. Vamos ver o resultado...

In [28]:
uni

{'id': '2534508389',
 'nome': 'Apartamento com 2 Quartos à Venda, 124m²',
 'ende': 'Alameda Tietê, 312 - Cerqueira César, São Paulo - SP',
 'metragem': 124,
 'preco': 1900000,
 'desc': '124   m²     2   Quartos     3   Banheiros     2   Vagas'}

In [52]:
imóveis

[{'id': '2537788782',
  'nome': 'Apartamento com 2 Quartos à Venda, 86m²',
  'ende': 'Rua Demóstenes, 606 - Campo Belo, São Paulo - SP',
  'metragem': 86,
  'preco': 1200000,
  'desc': '86   m²     2   Quartos     2   Banheiros     2   Vagas'},
 {'id': '2537810881',
  'nome': 'Apartamento com 2 Quartos à Venda, 95m²',
  'ende': 'Avenida Prefeito Dulcídio Cardoso, 1350 - Barra da Tijuca, Rio de Janeiro - RJ',
  'metragem': 95,
  'preco': 1250000,
  'desc': '95   m²     2   Quartos     2   Banheiros     1   Vaga'},
 {'id': '2533629678',
  'nome': 'Casa com 4 Quartos à Venda, 110m²',
  'ende': 'Rua Santo Antônio, 635 - Centro, São João de Meriti - RJ',
  'metragem': 110,
  'preco': 300000,
  'desc': '110   m²     4   Quartos     3   Banheiros     1   Vaga'},
 {'id': '2537096837',
  'nome': 'Apartamento com 3 Quartos à Venda, 253m²',
  'ende': 'Rua Pedro Doll, 391 - Santana, São Paulo - SP',
  'metragem': 253,
  'preco': 1950000,
  'desc': '253   m²     3   Quartos     5   Banheiros     3 

Agora converterei minha lista de dicts para um DataFrame chamado `todosOsImoveis`. Para tal, crio um DataFrame vazio nomeando as colunas com as chaves de um dict `uni`.

In [30]:
# Crio um DataFrame vazio cujas colunas são as chaves de 1 unidade:
todosOsImoveis=pd.DataFrame(columns=uni.keys())
todosOsImoveis

,id,nome,ende,metragem,preco,desc


Com o esqueleto do DataFrame pronto, adiciono nele todos os meus dicts (imóveis).

In [31]:
todosOsImoveis=todosOsImoveis.append(imóveis)

todosOsImoveis.head()

,id,nome,ende,metragem,preco,desc
0,2536709454,"Apartamento com 3 Quartos à Venda, 201m²","Alameda Jaú, 186 - Jardim Paulista, São Paulo ...",201,2140000,201 m² 3 Quartos 4 Banheiros ...
1,2537810881,"Apartamento com 2 Quartos à Venda, 95m²","Avenida Prefeito Dulcídio Cardoso, 1350 - Barr...",95,1250000,95 m² 2 Quartos 2 Banheiros ...
2,2535734789,"Apartamento com Quarto à Venda, 48m²","Rua Júlio D'Acia Barreto, 72 - Carvoeira, Flor...",48,520000,48 m² 1 Quarto 2 Banheiros 1...
3,2533629678,"Casa com 4 Quartos à Venda, 110m²","Rua Santo Antônio, 635 - Centro, São João de M...",110,300000,110 m² 4 Quartos 3 Banheiros ...
4,2536441425,"Casa com 3 Quartos à Venda, 99m²","Rua Manoel Borges Leão, 110 - Sabará III, Lond...",99,345000,99 m² 3 Quartos 3 Banheiros ...


Agora em Pandas, vamos dar uma otimizadinha básica convertendo algumas colunas textuais para tipos numéricos.

In [32]:
todosOsImoveis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720 entries, 0 to 719
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        720 non-null    object
 1   nome      720 non-null    object
 2   ende      720 non-null    object
 3   metragem  720 non-null    object
 4   preco     720 non-null    object
 5   desc      720 non-null    object
dtypes: object(6)
memory usage: 39.4+ KB


Como ficaram os tipos de nossas colunas...

In [33]:
todosOsImoveis=todosOsImoveis.convert_dtypes()

In [34]:
todosOsImoveis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720 entries, 0 to 719
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        720 non-null    string
 1   nome      720 non-null    string
 2   ende      720 non-null    string
 3   metragem  720 non-null    Int64 
 4   preco     720 non-null    Int64 
 5   desc      720 non-null    string
dtypes: Int64(2), string(4)
memory usage: 40.8 KB


Por que a coluna `id` não foi convertida para `Int64` como as outras? Vamos forçar:

In [35]:
todosOsImoveis['id']=pd.to_numeric(todosOsImoveis['id'])

In [36]:
todosOsImoveis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720 entries, 0 to 719
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        720 non-null    int64 
 1   nome      720 non-null    string
 2   ende      720 non-null    string
 3   metragem  720 non-null    Int64 
 4   preco     720 non-null    Int64 
 5   desc      720 non-null    string
dtypes: Int64(2), int64(1), string(3)
memory usage: 40.8 KB


Seta o índice para o ID do imóvel

In [37]:
todosOsImoveis.set_index(keys='id', inplace=True)

Eis o nosso DataFrame pronto:

In [38]:
todosOsImoveis.head()

,nome,ende,metragem,preco,desc
id,,,,,
2536709454,"Apartamento com 3 Quartos à Venda, 201m²","Alameda Jaú, 186 - Jardim Paulista, São Paulo ...",201,2140000,201 m² 3 Quartos 4 Banheiros ...
2537810881,"Apartamento com 2 Quartos à Venda, 95m²","Avenida Prefeito Dulcídio Cardoso, 1350 - Barr...",95,1250000,95 m² 2 Quartos 2 Banheiros ...
2535734789,"Apartamento com Quarto à Venda, 48m²","Rua Júlio D'Acia Barreto, 72 - Carvoeira, Flor...",48,520000,48 m² 1 Quarto 2 Banheiros 1...
2533629678,"Casa com 4 Quartos à Venda, 110m²","Rua Santo Antônio, 635 - Centro, São João de M...",110,300000,110 m² 4 Quartos 3 Banheiros ...
2536441425,"Casa com 3 Quartos à Venda, 99m²","Rua Manoel Borges Leão, 110 - Sabará III, Lond...",99,345000,99 m² 3 Quartos 3 Banheiros ...


## Grava o DataFrame inteiro num Banco de Dados

Este é o código para abrir ou criar um aquivo SQLite chamado `VivaReal.db`, na tabela `venda`. Para usar outro banco, como MariaDB, Oracle, DB2, altere esta célula e use a biblioteca SQL Alchemy com os drivers corretos. Todo o resto do código funcionará igual.

In [39]:
dbfile = 'VivaReal.db'
tabela = 'venda'

db = sqlite3.connect(dbfile)

Determina os tipos de cada coluna SQL baseado nos tipos das colunas Pandas:

In [40]:
sqlDataTypes={}

for c in todosOsImoveis.columns:
    if todosOsImoveis[c].dtype.kind == 'i':
        sqlDataTypes[c]='INTEGER'
    elif todosOsImoveis[c].dtype.kind == 'f':
        sqlDataTypes[c]='REAL'
    else:
        sqlDataTypes[c]='TEXT'

sqlDataTypes

{'nome': 'TEXT',
 'ende': 'TEXT',
 'metragem': 'INTEGER',
 'preco': 'INTEGER',
 'desc': 'TEXT'}

Escreve na tabela

In [41]:
todosOsImoveis.to_sql(tabela, index=True, if_exists='replace', dtype=sqlDataTypes, con=db)
# if_exists='append'

Efetiva as operações no DB e fecha o arquivo

In [42]:
db.commit()
db.close()

## Checa os dados no DB

In [43]:
dbfile = 'VivaReal.db'
tabela = 'venda'

db = sqlite3.connect(dbfile)

imoveisCaros = pd.read_sql_query(f'select * from "{tabela}" where preco>600000', db)

imoveisCaros

,id,nome,ende,metragem,preco,desc
0,2536709454,"Apartamento com 3 Quartos à Venda, 201m²","Alameda Jaú, 186 - Jardim Paulista, São Paulo ...",201,2140000,201 m² 3 Quartos 4 Banheiros ...
1,2537810881,"Apartamento com 2 Quartos à Venda, 95m²","Avenida Prefeito Dulcídio Cardoso, 1350 - Barr...",95,1250000,95 m² 2 Quartos 2 Banheiros ...
2,2533469793,"Apartamento com 3 Quartos à Venda, 151m²","Praça Quinze de Novembro, 84 - Cambuí, Campina...",151,895000,151 m² 3 Quartos 2 Banheiros ...
3,2535182780,"Apartamento com 4 Quartos à Venda, 118m²","Rua Constante Ramos, 82 - Copacabana, Rio de J...",118,950000,118 m² 4 Quartos 2 Banheiros ...
4,2534494985,"Apartamento com 3 Quartos à Venda, 128m²","Rua Bartira, 1099 - Perdizes, São Paulo - SP",128,870000,128 m² 3 Quartos 2 Banheiros ...
...,...,...,...,...,...,...
410,2536991679,"Apartamento com Quarto à Venda, 42m²","Rua Paula Ney - Vila Mariana, São Paulo - SP",42,780000,42 m² 1 Quarto 1 Banheiro 1 ...
411,2533248866,"Apartamento com 3 Quartos à Venda, 104m²","Rua 231, 10 - Meia Praia, Itapema - SC",104,1050000,104 m² 3 Quartos 2 Banheiros ...
412,2530411215,"Apartamento com 4 Quartos à Venda, 142m²","Alameda do Remo - Riviera de São Lourenço, Ber...",142,2300000,142 m² 4 Quartos 3 Banheiros ...
413,2535958495,"Apartamento com 3 Quartos à Venda, 100m²","Rua Mariz e Barros, 58 - Icaraí, Niterói - RJ",100,1100000,100 m² 3 Quartos 3 Banheiros ...


## Sincronização Diária

Para manter os dados em dia, este procedimento deve ser executado diariamente.

No final da extração comparo todos os imóveis da minha base histórica com todos os imóveis que acabei de extrair novamente. ID por ID. É por isso que é tão importante se prender aos IDs. É neste momento que detecto qual imóvel sumiu da base, qual apareceu, qual mudou de preço ou descrição.

* Imóveis que estão na minha base histórica mas não estão na última extração serão marcados como deletados ou “unlisted”.
* Imóveis que não estavam na minha base histórica e que acabaram de aparecer na última extração serão marcados como novos.
* Imóveis que aprentam preços diferentes entre a base histórica e a última extração devem ter essa alteração registrada em outra tabela. É interessante manter esse histórico de alteração de preços de imóveis.

Certamente nossa modelagem simples de tabela não suporta registrar a mudança de preços. **Fica de lição** de casa criar uma tabela separada de preços de imóveis para que se possa manter um histórico.

## Exercício

Melhorar o código e extrair também a lista de amenidades (elevador, piscina, churrasqueira etc)